In [14]:
import json
import requests
from bs4 import BeautifulSoup
import re
from time import sleep
from tqdm import tqdm
import pandas as pd
import os

### Header 정보

https://goodclickad.tistory.com/14 여기에서 https://www.coupang.com/vp/product/reviews?productId=133071001 이 링크로 연결하면 쿠키를 받을 수 있다.


헤더에 보내줘야 하는 데이터
1. cookie 


    아래 정보들은 set-cookie로 서버에서 전달해주는 값들이다. 최소 이건 갖고있어야 서버에서 응답해줌.  
    - sid  
    - CSID  
    - CUPT  
    - member_srl  
    - ILOGIN  
    - x-coupang-accept-language  
    - X-CP-PT-locale  
    - _abck  
    - _bm-sv    
    

2. referer  
3. user-agent

참고사이트  
https://goodclickad.tistory.com/14
https://www.coupang.com/vp/product/reviews?productId=133071001

In [4]:
# headers 정의
headers = {
    'cookie' : 'sid=c716d9887c55473fa97f3ecb5d96c195cdb31194; CSID=2iI6EXj3jsO1mNji2tbi-Rd2fSWwFHWFCfXRrZXJOizU_Ifjq30iky1ifyXkxUhpowg0OyfN1jNFTC3aWvVQjOpcvHRUS23NJQRAn33QfIvT; CUPT=7Tc1Z6TaXb1Ak43o7Sl2xgVbbKvJQ4A79OWXGXSi4KG2ZbzjHeFF_sNLe1zW0XHIhqzgaYFi2dyXhMU-fKugOr-Hb8gyZuRW9-3xxqbni4axO8o3Ia8UEnFjUhkmiPfcDsGYORYjrNHKITVrKAch4LDjlMktK7DQjD_MiKl72JDtUJBvFLSlfIx5Q5StYeZM_vcXVWFGtjE98-smBAuQuFUh00qXxtxLE7J5jIOOnfbSklc88NW6tns; member_srl=112475615; ILOGIN=Y;x-coupang-accept-language=ko_KR; X-CP-PT-locale=ko_KR; _abck=A5922A5151EFA8F2810A914AA1910EB2~0~YAAQhw3VFyTIU66FAQAAgwFbyAkxchATCfz0Ox/0zI1JKmH0oWHjkDONL9RVw8s9GUzeKL514MBNAcPdZ8babePGeJ3RuVzj6d3gMwL0KI+LLH3NOpDDyZWqnIlcpiOjj8Wf5r/UNoivJs8nW14FxBRL0xB8muieQWi91Z3doS8rutWqW5/rJv+enw0iAf5Hd7nuAYuArtTDQB8E8OCCz8CY16CyU9lJLeuH/7df7BL9wXebMJZGybzUu6Sg4601/GuEOi+BVdPgKyXR2x6ybiuOd1bG7mRYqAn0ssOR845UIiqTkGcLjz26qlJw9K/cvwHBJ2mD7GKxJnJ1N6P4Cqf8fOYLGLg6dSh2z+Kn6gHpM43JsRqCrVuO/0tcde5i8vXd3Ljv0t7YGWeq2Q==~-1~-1~-1; bm_sv=3CB077DCB31C697760014054E8D5518E~YAAQhw3VFyXIU66FAQAAgwFbyBKPxnu1HD5Vp5HpyIEgsnZlIvuqUNjs8Wr1OpnB0ew8eyb80bJFVVS3QIdbN1oUWzBFws69htQ/85fizSClZfW5Uy3MSZxLVZ0MIhrQn4tzcNrL2r+lUb6IR9FPzjscmq4UVyTSPGG21TgPBrIdCq935/8sbpdr4aIYV3aafygvOir7Suy+2vuMUrmWoEaJGJ5GeHXY0kEqBZCph7EuMIzaOAvJ5tEG1DI6WF4DCDI1~1',
    'referer' : "https://goodclickad.tistory.com/13",
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Whale/3.18.154.7 Safari/537.36'
}

In [38]:
# test
url = 'https://www.coupang.com/vp/product/reviews?productId=133071001'

headers = {
    'cookie' : 'sid=c716d9887c55473fa97f3ecb5d96c195cdb31194; CSID=2iI6EXj3jsO1mNji2tbi-Rd2fSWwFHWFCfXRrZXJOizU_Ifjq30iky1ifyXkxUhpowg0OyfN1jNFTC3aWvVQjOpcvHRUS23NJQRAn33QfIvT; CUPT=7Tc1Z6TaXb1Ak43o7Sl2xgVbbKvJQ4A79OWXGXSi4KG2ZbzjHeFF_sNLe1zW0XHIhqzgaYFi2dyXhMU-fKugOr-Hb8gyZuRW9-3xxqbni4axO8o3Ia8UEnFjUhkmiPfcDsGYORYjrNHKITVrKAch4LDjlMktK7DQjD_MiKl72JDtUJBvFLSlfIx5Q5StYeZM_vcXVWFGtjE98-smBAuQuFUh00qXxtxLE7J5jIOOnfbSklc88NW6tns; member_srl=112475615; ILOGIN=Y;x-coupang-accept-language=ko_KR; X-CP-PT-locale=ko_KR; _abck=A5922A5151EFA8F2810A914AA1910EB2~0~YAAQhw3VFyTIU66FAQAAgwFbyAkxchATCfz0Ox/0zI1JKmH0oWHjkDONL9RVw8s9GUzeKL514MBNAcPdZ8babePGeJ3RuVzj6d3gMwL0KI+LLH3NOpDDyZWqnIlcpiOjj8Wf5r/UNoivJs8nW14FxBRL0xB8muieQWi91Z3doS8rutWqW5/rJv+enw0iAf5Hd7nuAYuArtTDQB8E8OCCz8CY16CyU9lJLeuH/7df7BL9wXebMJZGybzUu6Sg4601/GuEOi+BVdPgKyXR2x6ybiuOd1bG7mRYqAn0ssOR845UIiqTkGcLjz26qlJw9K/cvwHBJ2mD7GKxJnJ1N6P4Cqf8fOYLGLg6dSh2z+Kn6gHpM43JsRqCrVuO/0tcde5i8vXd3Ljv0t7YGWeq2Q==~-1~-1~-1; bm_sv=3CB077DCB31C697760014054E8D5518E~YAAQhw3VFyXIU66FAQAAgwFbyBKPxnu1HD5Vp5HpyIEgsnZlIvuqUNjs8Wr1OpnB0ew8eyb80bJFVVS3QIdbN1oUWzBFws69htQ/85fizSClZfW5Uy3MSZxLVZ0MIhrQn4tzcNrL2r+lUb6IR9FPzjscmq4UVyTSPGG21TgPBrIdCq935/8sbpdr4aIYV3aafygvOir7Suy+2vuMUrmWoEaJGJ5GeHXY0kEqBZCph7EuMIzaOAvJ5tEG1DI6WF4DCDI1~1',
    'referer' : "https://aytekin.tistory.com/118",
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'
}
res = requests.get(url,headers=headers)
res.status_code

403

### Category ID 구하기

In [81]:
# 카테고리 ID구하는 함수 정의
def get_category_id(id):
    ''' id에 여성패션 - 의류 와 같은 카테고리 아이디를 id를 넣어주면 됩니다.'''

    base_url = 'https://www.coupang.com/np/categories/{}'
    headers = {
    'cookie' : 'sid=c716d9887c55473fa97f3ecb5d96c195cdb31194; CSID=2iI6EXj3jsO1mNji2tbi-Rd2fSWwFHWFCfXRrZXJOizU_Ifjq30iky1ifyXkxUhpowg0OyfN1jNFTC3aWvVQjOpcvHRUS23NJQRAn33QfIvT; CUPT=7Tc1Z6TaXb1Ak43o7Sl2xgVbbKvJQ4A79OWXGXSi4KG2ZbzjHeFF_sNLe1zW0XHIhqzgaYFi2dyXhMU-fKugOr-Hb8gyZuRW9-3xxqbni4axO8o3Ia8UEnFjUhkmiPfcDsGYORYjrNHKITVrKAch4LDjlMktK7DQjD_MiKl72JDtUJBvFLSlfIx5Q5StYeZM_vcXVWFGtjE98-smBAuQuFUh00qXxtxLE7J5jIOOnfbSklc88NW6tns; member_srl=112475615; ILOGIN=Y;x-coupang-accept-language=ko_KR; X-CP-PT-locale=ko_KR; _abck=A5922A5151EFA8F2810A914AA1910EB2~0~YAAQhw3VFyTIU66FAQAAgwFbyAkxchATCfz0Ox/0zI1JKmH0oWHjkDONL9RVw8s9GUzeKL514MBNAcPdZ8babePGeJ3RuVzj6d3gMwL0KI+LLH3NOpDDyZWqnIlcpiOjj8Wf5r/UNoivJs8nW14FxBRL0xB8muieQWi91Z3doS8rutWqW5/rJv+enw0iAf5Hd7nuAYuArtTDQB8E8OCCz8CY16CyU9lJLeuH/7df7BL9wXebMJZGybzUu6Sg4601/GuEOi+BVdPgKyXR2x6ybiuOd1bG7mRYqAn0ssOR845UIiqTkGcLjz26qlJw9K/cvwHBJ2mD7GKxJnJ1N6P4Cqf8fOYLGLg6dSh2z+Kn6gHpM43JsRqCrVuO/0tcde5i8vXd3Ljv0t7YGWeq2Q==~-1~-1~-1; bm_sv=3CB077DCB31C697760014054E8D5518E~YAAQhw3VFyXIU66FAQAAgwFbyBKPxnu1HD5Vp5HpyIEgsnZlIvuqUNjs8Wr1OpnB0ew8eyb80bJFVVS3QIdbN1oUWzBFws69htQ/85fizSClZfW5Uy3MSZxLVZ0MIhrQn4tzcNrL2r+lUb6IR9FPzjscmq4UVyTSPGG21TgPBrIdCq935/8sbpdr4aIYV3aafygvOir7Suy+2vuMUrmWoEaJGJ5GeHXY0kEqBZCph7EuMIzaOAvJ5tEG1DI6WF4DCDI1~1',
    'referer' : "https://goodclickad.tistory.com/14",
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Whale/3.18.154.7 Safari/537.36'
    }

    response = requests.get(base_url.format(id), headers=headers)
    print(response)

    soup = BeautifulSoup(response.text,'html.parser')
    ids = soup.find('map').find_all('area')

    category_li = []
    
    for id_soup in tqdm(ids):
        
        p = re.compile('[0-9]')
        category_id = "".join(p.findall(id_soup['href']))[:-1]

        title_res = requests.get(base_url.format(category_id),headers=headers)
        title_soup = BeautifulSoup(title_res.text,'html.parser')
        
        try:
            category_name = title_soup.find('h3',{'class':'newcx-product-list-title'}).get_text(strip=True)
        except:
            category_name = None
        try:
            big_category_name = soup.find('div',class_='search-result').get_text(strip=True)
        except:
            big_category_name = None
        try:
            link = base_url.format(category_id)
        except:
            link = None

        dic = {}
        dic['big_category_name'] = big_category_name
        dic['category_name'] = category_name
        dic['category_id'] = category_id
        dic['link'] = link
        category_li.append(dic)
        sleep(1)

    print('총 {}의 카테고리 id {}개를 수집하였습니다. '.format(big_category_name, len(category_li)))
    return category_li

# get_category_id(498775)

In [82]:
# 여성 - 의류, 여성 - 속옷/잠옷, 남성 - 의류, 남성 - 속옷/잠옷

big_category_id_li = [
    498704,
    498775,
    498917,
    498962,
]

category_id_li = []

for id in big_category_id_li:
    temp = get_category_id(id)
    category_id_li.append(temp)

<Response [200]>


100%|██████████| 18/18 [00:41<00:00,  2.28s/it]


총 쿠팡 홈여성패션의류의 카테고리 id 18개를 수집하였습니다. 
<Response [200]>


100%|██████████| 15/15 [00:31<00:00,  2.09s/it]


총 쿠팡 홈여성패션속옷/잠옷의 카테고리 id 15개를 수집하였습니다. 
<Response [200]>


100%|██████████| 15/15 [00:34<00:00,  2.30s/it]


총 쿠팡 홈남성패션의류의 카테고리 id 15개를 수집하였습니다. 
<Response [200]>


100%|██████████| 10/10 [00:22<00:00,  2.26s/it]

총 쿠팡 홈남성패션속옷/잠옷의 카테고리 id 10개를 수집하였습니다. 


In [95]:
# 공용의류
    
common_clothes=[
    502994,
    502995,
    502998,
    502999,
    503000,
    503001,
    503002,
    503006,
    503010,
    503013,
    509288
]

com_category_li = []

for id in tqdm(common_clothes):
    url = 'https://www.coupang.com/np/categories/{}'.format(id)
    res = requests.get(url,headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')

    try:
        category_name = soup.find('h3',{'class':'newcx-product-list-title'}).get_text(strip=True)
    except:
        category_name = None
    try:
        big_category_name = soup.find('div',class_='search-result').get_text(strip=True)
    except:
        big_category_name = None        
    category_id = id
    try:
        link = url.format(id)
    except:
        link = None

    dic = {}
    dic['big_category_name'] = big_category_name
    dic['category_name'] = category_name
    dic['category_id'] = category_id
    dic['link'] = link
    
    com_category_li.append(dic)

print('총 {}의 카테고리 id {}개를 수집하였습니다. '.format(big_category_name, len(com_category_li)))


100%|██████████| 11/11 [00:12<00:00,  1.10s/it]

총 쿠팡 홈남녀 공용 의류빅사이즈의 카테고리 id 11개를 수집하였습니다. 


In [96]:
# category id 합치기
category_ids = pd.DataFrame(sum(category_id_li,[]) + com_category_li)
category_ids.head()

,big_category_name,category_name,category_id,link
0,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705
1,쿠팡 홈여성패션의류,맨투맨/후드티,498706,https://www.coupang.com/np/categories/498706
2,쿠팡 홈여성패션의류,블라우스/셔츠,498709,https://www.coupang.com/np/categories/498709
3,쿠팡 홈여성패션의류,원피스,498712,https://www.coupang.com/np/categories/498712
4,쿠팡 홈여성패션의류,정장/세트류,498713,https://www.coupang.com/np/categories/498713


In [97]:
# json으로 크롤링 된 데이터 저장
category_ids.to_json('category_ids.json', force_ascii=False, orient='records')

In [17]:
# json으로 크롤링 된 데이터 가져오기
with open(r'data\category_ids.json', encoding='utf-8') as f:
    temp = json.load(f)
category_ids = pd.DataFrame(temp)
category_ids

,big_category_name,category_name,category_id,link
0,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705
1,쿠팡 홈여성패션의류,맨투맨/후드티,498706,https://www.coupang.com/np/categories/498706
2,쿠팡 홈여성패션의류,블라우스/셔츠,498709,https://www.coupang.com/np/categories/498709
3,쿠팡 홈여성패션의류,원피스,498712,https://www.coupang.com/np/categories/498712
4,쿠팡 홈여성패션의류,정장/세트류,498713,https://www.coupang.com/np/categories/498713
...,...,...,...,...
64,쿠팡 홈남녀 공용 의류니트류/조끼,니트류/조끼,503002,https://www.coupang.com/np/categories/503002
65,쿠팡 홈남녀 공용 의류아우터,아우터,503006,https://www.coupang.com/np/categories/503006
66,쿠팡 홈남녀 공용 의류테마의류,테마의류,503010,https://www.coupang.com/np/categories/503010
67,쿠팡 홈남녀 공용 의류커플룩/패밀리룩,커플룩/패밀리룩,503013,https://www.coupang.com/np/categories/503013


---

### 카테고리별 크롤링

카테고리 분류

1. 후드/집업류(남,여,공)
2. 티셔츠(남,여,공)
3. 맨투맨/후드티(남,여,공)
4. 셔츠/남방(남) + 셔츠(공)
5. 셔츠/블라우스(여)
6. 바지(남,공)
7. 스커트/치마(여)
8. 니트류/조끼(남,여,공)
9. 트레이닝복(남,여,공)
10. 아우터(남,여,공)

#### 1. 후드/집업류(남,여,공)

In [7]:
big_category_name = ['쿠팡 홈여성패션의류','쿠팡 홈남성패션의류','쿠팡 홈남여공용의류']
category_id = [498724, 498929, 503001]
category_ids = pd.DataFrame({
    'big_category_name' : big_category_name,
    'category_name' : '후드집업/집업류',
    'category_id':category_id,
    })
category_ids

,big_category_name,category_name,category_id
0,쿠팡 홈여성패션의류,후드집업/집업류,498724
1,쿠팡 홈남성패션의류,후드집업/집업류,498929
2,쿠팡 홈남여공용의류,후드집업/집업류,503001


In [12]:
base_url = 'https://www.coupang.com/np/categories/{}'

data = []

for index, row in tqdm(category_ids.iterrows()):
    category_id = row['category_id']
    
    for page in range(1,11):
        print('[{}-{}] category의 {}page 상품 ID 크롤링 시작'.format(row['big_category_name'],row['category_name'],page))
        params = {
            'listSize' : 120,
            'page' : page,
            'sorter' : 'saleCountDesc'
        }
        try:
            res = requests.get(base_url.format(category_id), headers=headers, params=params)
            soup = BeautifulSoup(res.text,'html.parser')
            prod_id_li = eval(soup.find('ul', id='productList')['data-products'])['indexes']

            for prod_id in prod_id_li:
                dic = {}
                try:
                    dic['big_category_name'] = row['big_category_name']
                except:
                    dic['big_category_name'] = None
                try:
                    dic['category_name'] = row['category_name']
                except:
                    dic['category_name'] = None
                try:
                    dic['category_id'] = row['category_id']
                except:
                    dic['category_id'] = None
                try:
                    dic['link'] = row['link']
                except:
                    dic['link'] = None
                try:
                    dic['prod_id'] = prod_id
                except:
                    dic['prod_id'] = None
                data.append(dic)
        except:
            pass

0it [00:00, ?it/s]

[쿠팡 홈여성패션의류-후드집업/집업류] category의 1page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-후드집업/집업류] category의 2page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-후드집업/집업류] category의 3page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-후드집업/집업류] category의 4page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-후드집업/집업류] category의 5page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-후드집업/집업류] category의 6page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-후드집업/집업류] category의 7page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-후드집업/집업류] category의 8page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-후드집업/집업류] category의 9page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-후드집업/집업류] category의 10page 상품 ID 크롤링 시작


1it [00:13, 13.53s/it]

[쿠팡 홈남성패션의류-후드집업/집업류] category의 1page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-후드집업/집업류] category의 2page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-후드집업/집업류] category의 3page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-후드집업/집업류] category의 4page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-후드집업/집업류] category의 5page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-후드집업/집업류] category의 6page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-후드집업/집업류] category의 7page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-후드집업/집업류] category의 8page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-후드집업/집업류] category의 9page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-후드집업/집업류] category의 10page 상품 ID 크롤링 시작


2it [00:27, 13.89s/it]

[쿠팡 홈남여공용의류-후드집업/집업류] category의 1page 상품 ID 크롤링 시작
[쿠팡 홈남여공용의류-후드집업/집업류] category의 2page 상품 ID 크롤링 시작
[쿠팡 홈남여공용의류-후드집업/집업류] category의 3page 상품 ID 크롤링 시작
[쿠팡 홈남여공용의류-후드집업/집업류] category의 4page 상품 ID 크롤링 시작
[쿠팡 홈남여공용의류-후드집업/집업류] category의 5page 상품 ID 크롤링 시작
[쿠팡 홈남여공용의류-후드집업/집업류] category의 6page 상품 ID 크롤링 시작
[쿠팡 홈남여공용의류-후드집업/집업류] category의 7page 상품 ID 크롤링 시작
[쿠팡 홈남여공용의류-후드집업/집업류] category의 8page 상품 ID 크롤링 시작
[쿠팡 홈남여공용의류-후드집업/집업류] category의 9page 상품 ID 크롤링 시작
[쿠팡 홈남여공용의류-후드집업/집업류] category의 10page 상품 ID 크롤링 시작


3it [00:41, 13.77s/it]


In [13]:
df = pd.DataFrame(data)
df
# 3239개 상품

,big_category_name,category_name,category_id,link,prod_id
0,쿠팡 홈여성패션의류,후드집업/집업류,498724,None,1592283662
1,쿠팡 홈여성패션의류,후드집업/집업류,498724,None,2311426786
2,쿠팡 홈여성패션의류,후드집업/집업류,498724,None,6086676051
3,쿠팡 홈여성패션의류,후드집업/집업류,498724,None,6702408280
4,쿠팡 홈여성패션의류,후드집업/집업류,498724,None,6828612379
...,...,...,...,...,...
3234,쿠팡 홈남여공용의류,후드집업/집업류,503001,None,4757877011
3235,쿠팡 홈남여공용의류,후드집업/집업류,503001,None,6056547508
3236,쿠팡 홈남여공용의류,후드집업/집업류,503001,None,6957027751
3237,쿠팡 홈남여공용의류,후드집업/집업류,503001,None,6073996296


In [14]:
# json으로 크롤링 된 데이터 저장하기
df.to_json('product_ids.json', force_ascii=False, orient='records')

In [ ]:
# json으로 크롤링 된 데이터 가져오기
with open('product_ids.json', encoding='utf-8') as f:
    temp = json.load(f)
temp = pd.DataFrame(temp)
temp

In [15]:
base_url = 'https://www.coupang.com/vp/product/reviews'
review = []

for index, row in tqdm(df.iterrows()):

    big_category_name = row['big_category_name']
    category_name = row['category_name']
    category_id = row['category_id']
    category_link = row['link']
    prod_id = row['prod_id']

    for page in tqdm(range(1,21)):
        params = {
            'productId' : prod_id,
            'page' : page,
            'size' : 30,
            'sortBy' : 'ORDER_SCORE_ASC',
            'viRoleCode' : 3,
            'ratingSummary' : 'true'
        }
        
        res = requests.get(base_url,headers=headers, params=params)
        soup = BeautifulSoup(res.text, 'html.parser')
        datas = soup.find_all('article',{'class':'sdp-review__article__list js_reviewArticleReviewList'})
        for data in datas:
            dic = {}
            try:
                dic['review'] = data.find('div',{'class':'sdp-review__article__list__review__content js_reviewArticleContent'}).get_text(strip=True)
            except:
                continue
            try:
                dic['review_headline'] = data.find('div',{'class':'sdp-review__article__list__headline'}).get_text(strip=True)
            except:
                dic['review_headline'] = None            
            try:
                dic['reviewer'] = data.find('div',{'class':'sdp-review__article__list__info__user'}).get_text(strip=True)
            except:
                dic['reviewer'] = None
            try:
                dic['prod_name'] = data.find('div',{'class':'sdp-review__article__list__info__product-info__name'}).get_text(strip=True)
            except:
                dic['prod_name'] = None
            try:
                dic['prod_id'] = prod_id
            except:
                dic['prod_id'] = None
            try:
                dic['prod_link'] = 'https://www.coupang.com/vp/products/{}'.format(prod_id)
            except:
                dic['prod_link'] = None
            try:
                dic['prod_star_score'] = data.find('div',{'class':'sdp-review__article__list__info__product-info__star-orange js_reviewArticleRatingValue'})['data-rating']
            except:
                dic['prod_star_score'] = None
            try:
                dic['prod_reg_date'] = data.find('div',{'class':'sdp-review__article__list__info__product-info__reg-date'}).get_text(strip=True)
            except:
                dic['prod_reg_date'] = None
            try:
                dic['attach_count'] = len(data.find_all('div',{'class':'sdp-review__article__list__attachment__list'}))
            except:
                dic['attach_count'] = None
            try:
                dic['help_count'] = data.find('span',{'class':'js_reviewArticleHelpfulCount'}).get_text()
            except:
                dic['help_count'] = None
            try:
                dic['big_category_name'] = big_category_name
            except:
                dic['big_category_name'] = None
            try:
                dic['category_name'] = category_name
            except:
                dic['category_name'] = None
            try:
                dic['category_id'] = category_id
            except:
                dic['category_id'] = None
            try:
                dic['category_link'] = category_link
            except:
                dic['category_link'] = None
            review.append(dic)

# 크롤링 된 데이터 저장하기
review_df = pd.DataFrame(review)
review_df.to_csv(r'C:\Users\Aytekin\Desktop\review_reliability\hoodzip_up_model\hoodzipup_review_data.csv', encoding='utf-8')

100%|██████████| 20/20 [00:03<00:00,  6.18it/s]
3239it [2:29:58,  2.78s/it]


ValueError: No engine for filetype: 'csv'

#### 2. 티셔츠(남,여,공)

In [26]:
cate_name = '티셔츠'
cate_ids_Tshirts = category_ids[category_ids['category_name'] == cate_name]
cate_ids_Tshirts

,big_category_name,category_name,category_id,link
0,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705
33,쿠팡 홈남성패션의류,티셔츠,498918,https://www.coupang.com/np/categories/498918
58,쿠팡 홈남녀 공용 의류티셔츠,티셔츠,502994,https://www.coupang.com/np/categories/502994


In [24]:
# 상품 ID 크롤링

base_url = 'https://www.coupang.com/np/categories/{}'

data = []

for index, row in tqdm(cate_ids_Tshirts.iterrows()):
    category_id = row['category_id']
    
    for page in range(1,11):
        print('[{}-{}] category의 {}page 상품 ID 크롤링 시작'.format(row['big_category_name'],row['category_name'],page))
        params = {
            'listSize' : 120,
            'page' : page,
            'sorter' : 'saleCountDesc'
        }
        try:
            res = requests.get(base_url.format(category_id), headers=headers, params=params)
            soup = BeautifulSoup(res.text,'html.parser')
            prod_id_li = eval(soup.find('ul', id='productList')['data-products'])['indexes']

            for prod_id in prod_id_li:
                dic = {}
                try:
                    dic['big_category_name'] = row['big_category_name']
                except:
                    dic['big_category_name'] = None
                try:
                    dic['category_name'] = row['category_name']
                except:
                    dic['category_name'] = None
                try:
                    dic['category_id'] = row['category_id']
                except:
                    dic['category_id'] = None
                try:
                    dic['link'] = row['link']
                except:
                    dic['link'] = None
                try:
                    dic['prod_id'] = prod_id
                except:
                    dic['prod_id'] = None
                data.append(dic)
        except:
            pass

0it [00:00, ?it/s]

[쿠팡 홈여성패션의류-티셔츠] category의 1page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-티셔츠] category의 2page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-티셔츠] category의 3page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-티셔츠] category의 4page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-티셔츠] category의 5page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-티셔츠] category의 6page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-티셔츠] category의 7page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-티셔츠] category의 8page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-티셔츠] category의 9page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-티셔츠] category의 10page 상품 ID 크롤링 시작


1it [00:14, 14.01s/it]

[쿠팡 홈남성패션의류-티셔츠] category의 1page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-티셔츠] category의 2page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-티셔츠] category의 3page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-티셔츠] category의 4page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-티셔츠] category의 5page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-티셔츠] category의 6page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-티셔츠] category의 7page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-티셔츠] category의 8page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-티셔츠] category의 9page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-티셔츠] category의 10page 상품 ID 크롤링 시작


2it [00:27, 13.95s/it]

[쿠팡 홈남녀 공용 의류티셔츠-티셔츠] category의 1page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류티셔츠-티셔츠] category의 2page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류티셔츠-티셔츠] category의 3page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류티셔츠-티셔츠] category의 4page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류티셔츠-티셔츠] category의 5page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류티셔츠-티셔츠] category의 6page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류티셔츠-티셔츠] category의 7page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류티셔츠-티셔츠] category의 8page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류티셔츠-티셔츠] category의 9page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류티셔츠-티셔츠] category의 10page 상품 ID 크롤링 시작


3it [00:42, 14.22s/it]


In [25]:
prod_ids_Tshirts = pd.DataFrame(data)
prod_ids_Tshirts
# 3240개 상품

,big_category_name,category_name,category_id,link,prod_id
0,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705,6321766904
1,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705,6774464085
2,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705,6750877527
3,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705,4759461143
4,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705,4876745607
...,...,...,...,...,...
3235,쿠팡 홈남녀 공용 의류티셔츠,티셔츠,502994,https://www.coupang.com/np/categories/502994,6589985917
3236,쿠팡 홈남녀 공용 의류티셔츠,티셔츠,502994,https://www.coupang.com/np/categories/502994,6343917201
3237,쿠팡 홈남녀 공용 의류티셔츠,티셔츠,502994,https://www.coupang.com/np/categories/502994,6303905805
3238,쿠팡 홈남녀 공용 의류티셔츠,티셔츠,502994,https://www.coupang.com/np/categories/502994,6595467859


In [27]:
# json으로 크롤링 된 데이터 저장하기 - Tshirts
prod_ids_Tshirts.to_json('data/product_ids/product_ids_{}.json'.format(cate_name), force_ascii=False, orient='records')

In [29]:
# 리뷰크롤링
base_url = 'https://www.coupang.com/vp/product/reviews'
review = []

for index, row in tqdm(prod_ids_Tshirts.iterrows()):

    big_category_name = row['big_category_name']
    category_name = row['category_name']
    category_id = row['category_id']
    category_link = row['link']
    prod_id = row['prod_id']

    for page in tqdm(range(1,21)):
        params = {
            'productId' : prod_id,
            'page' : page,
            'size' : 30,
            'sortBy' : 'ORDER_SCORE_ASC',
            'viRoleCode' : 3,
            'ratingSummary' : 'true'
        }
        
        res = requests.get(base_url,headers=headers, params=params)
        soup = BeautifulSoup(res.text, 'html.parser')
        datas = soup.find_all('article',{'class':'sdp-review__article__list js_reviewArticleReviewList'})
        for data in datas:
            dic = {}
            try:
                dic['review'] = data.find('div',{'class':'sdp-review__article__list__review__content js_reviewArticleContent'}).get_text(strip=True)
            except:
                continue
            try:
                dic['review_headline'] = data.find('div',{'class':'sdp-review__article__list__headline'}).get_text(strip=True)
            except:
                dic['review_headline'] = None            
            try:
                dic['reviewer'] = data.find('div',{'class':'sdp-review__article__list__info__user'}).get_text(strip=True)
            except:
                dic['reviewer'] = None
            try:
                dic['prod_name'] = data.find('div',{'class':'sdp-review__article__list__info__product-info__name'}).get_text(strip=True)
            except:
                dic['prod_name'] = None
            try:
                dic['prod_id'] = prod_id
            except:
                dic['prod_id'] = None
            try:
                dic['prod_link'] = 'https://www.coupang.com/vp/products/{}'.format(prod_id)
            except:
                dic['prod_link'] = None
            try:
                dic['prod_star_score'] = data.find('div',{'class':'sdp-review__article__list__info__product-info__star-orange js_reviewArticleRatingValue'})['data-rating']
            except:
                dic['prod_star_score'] = None
            try:
                dic['prod_reg_date'] = data.find('div',{'class':'sdp-review__article__list__info__product-info__reg-date'}).get_text(strip=True)
            except:
                dic['prod_reg_date'] = None
            try:
                dic['attach_count'] = len(data.find_all('div',{'class':'sdp-review__article__list__attachment__list'}))
            except:
                dic['attach_count'] = None
            try:
                dic['help_count'] = data.find('span',{'class':'js_reviewArticleHelpfulCount'}).get_text()
            except:
                dic['help_count'] = None
            try:
                dic['big_category_name'] = big_category_name
            except:
                dic['big_category_name'] = None
            try:
                dic['category_name'] = category_name
            except:
                dic['category_name'] = None
            try:
                dic['category_id'] = category_id
            except:
                dic['category_id'] = None
            try:
                dic['category_link'] = category_link
            except:
                dic['category_link'] = None
            review.append(dic)

# 크롤링 된 데이터 저장하기
review_df = pd.DataFrame(review)
review_df.to_csv('data/reviews/reviews_{}.csv'.format(cate_name), encoding='utf-8')

100%|██████████| 20/20 [00:02<00:00,  8.04it/s]
3240it [2:52:56,  3.20s/it]


#### 3. 맨투맨/후드티(남,여,공)

In [36]:
cate_name = '맨투맨/후드티'
cate_ids_mantoman = category_ids[category_ids['category_name'] == cate_name]
if '/' in cate_name:
    cate_name = ''.join(cate_name.split('/'))
cate_ids_mantoman

,big_category_name,category_name,category_id,link
1,쿠팡 홈여성패션의류,맨투맨/후드티,498706,https://www.coupang.com/np/categories/498706
34,쿠팡 홈남성패션의류,맨투맨/후드티,498919,https://www.coupang.com/np/categories/498919
59,쿠팡 홈남녀 공용 의류맨투맨/후드티,맨투맨/후드티,502995,https://www.coupang.com/np/categories/502995


In [37]:
# 상품 ID 크롤링

base_url = 'https://www.coupang.com/np/categories/{}'

data = []

for index, row in tqdm(cate_ids_mantoman.iterrows()):
    category_id = row['category_id']
    
    for page in range(1,11):
        print('[{}-{}] category의 {}page 상품 ID 크롤링 시작'.format(row['big_category_name'],row['category_name'],page))
        params = {
            'listSize' : 120,
            'page' : page,
            'sorter' : 'saleCountDesc'
        }
        try:
            res = requests.get(base_url.format(category_id), headers=headers, params=params)
            soup = BeautifulSoup(res.text,'html.parser')
            prod_id_li = eval(soup.find('ul', id='productList')['data-products'])['indexes']

            for prod_id in prod_id_li:
                dic = {}
                try:
                    dic['big_category_name'] = row['big_category_name']
                except:
                    dic['big_category_name'] = None
                try:
                    dic['category_name'] = row['category_name']
                except:
                    dic['category_name'] = None
                try:
                    dic['category_id'] = row['category_id']
                except:
                    dic['category_id'] = None
                try:
                    dic['link'] = row['link']
                except:
                    dic['link'] = None
                try:
                    dic['prod_id'] = prod_id
                except:
                    dic['prod_id'] = None
                data.append(dic)
        except:
            pass

prod_ids_mantoman = pd.DataFrame(data)
print(prod_ids_mantoman.shape[0])
# 3240개 상품

# json으로 크롤링 된 데이터 저장하기 - mantoman
prod_ids_mantoman.to_json('data/product_ids/product_ids_{}.json'.format(cate_name), force_ascii=False, orient='records')

0it [00:00, ?it/s]

[쿠팡 홈여성패션의류-맨투맨/후드티] category의 1page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-맨투맨/후드티] category의 2page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-맨투맨/후드티] category의 3page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-맨투맨/후드티] category의 4page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-맨투맨/후드티] category의 5page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-맨투맨/후드티] category의 6page 상품 ID 크롤링 시작


1it [00:00,  2.67it/s]

[쿠팡 홈여성패션의류-맨투맨/후드티] category의 7page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-맨투맨/후드티] category의 8page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-맨투맨/후드티] category의 9page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-맨투맨/후드티] category의 10page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-맨투맨/후드티] category의 1page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-맨투맨/후드티] category의 2page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-맨투맨/후드티] category의 3page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-맨투맨/후드티] category의 4page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-맨투맨/후드티] category의 5page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-맨투맨/후드티] category의 6page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-맨투맨/후드티] category의 7page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-맨투맨/후드티] category의 8page 상품 ID 크롤링 시작
[쿠팡 홈남성패션의류-맨투맨/후드티] category의 9page 상품 ID 크롤링 시작


2it [00:00,  2.61it/s]

[쿠팡 홈남성패션의류-맨투맨/후드티] category의 10page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류맨투맨/후드티-맨투맨/후드티] category의 1page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류맨투맨/후드티-맨투맨/후드티] category의 2page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류맨투맨/후드티-맨투맨/후드티] category의 3page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류맨투맨/후드티-맨투맨/후드티] category의 4page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류맨투맨/후드티-맨투맨/후드티] category의 5page 상품 ID 크롤링 시작


3it [00:01,  2.64it/s]

[쿠팡 홈남녀 공용 의류맨투맨/후드티-맨투맨/후드티] category의 6page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류맨투맨/후드티-맨투맨/후드티] category의 7page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류맨투맨/후드티-맨투맨/후드티] category의 8page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류맨투맨/후드티-맨투맨/후드티] category의 9page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류맨투맨/후드티-맨투맨/후드티] category의 10page 상품 ID 크롤링 시작


3it [00:01,  2.64it/s]

0


In [ ]:
# 리뷰크롤링
base_url = 'https://www.coupang.com/vp/product/reviews'
review = []

for index, row in tqdm(prod_ids_mantoman.iterrows()):

    big_category_name = row['big_category_name']
    category_name = row['category_name']
    category_id = row['category_id']
    category_link = row['link']
    prod_id = row['prod_id']

    for page in tqdm(range(1,21)):
        params = {
            'productId' : prod_id,
            'page' : page,
            'size' : 30,
            'sortBy' : 'ORDER_SCORE_ASC',
            'viRoleCode' : 3,
            'ratingSummary' : 'true'
        }
        
        res = requests.get(base_url,headers=headers, params=params)
        soup = BeautifulSoup(res.text, 'html.parser')
        datas = soup.find_all('article',{'class':'sdp-review__article__list js_reviewArticleReviewList'})
        for data in datas:
            dic = {}
            try:
                dic['review'] = data.find('div',{'class':'sdp-review__article__list__review__content js_reviewArticleContent'}).get_text(strip=True)
            except:
                continue
            try:
                dic['review_headline'] = data.find('div',{'class':'sdp-review__article__list__headline'}).get_text(strip=True)
            except:
                dic['review_headline'] = None            
            try:
                dic['reviewer'] = data.find('div',{'class':'sdp-review__article__list__info__user'}).get_text(strip=True)
            except:
                dic['reviewer'] = None
            try:
                dic['prod_name'] = data.find('div',{'class':'sdp-review__article__list__info__product-info__name'}).get_text(strip=True)
            except:
                dic['prod_name'] = None
            try:
                dic['prod_id'] = prod_id
            except:
                dic['prod_id'] = None
            try:
                dic['prod_link'] = 'https://www.coupang.com/vp/products/{}'.format(prod_id)
            except:
                dic['prod_link'] = None
            try:
                dic['prod_star_score'] = data.find('div',{'class':'sdp-review__article__list__info__product-info__star-orange js_reviewArticleRatingValue'})['data-rating']
            except:
                dic['prod_star_score'] = None
            try:
                dic['prod_reg_date'] = data.find('div',{'class':'sdp-review__article__list__info__product-info__reg-date'}).get_text(strip=True)
            except:
                dic['prod_reg_date'] = None
            try:
                dic['attach_count'] = len(data.find_all('div',{'class':'sdp-review__article__list__attachment__list'}))
            except:
                dic['attach_count'] = None
            try:
                dic['help_count'] = data.find('span',{'class':'js_reviewArticleHelpfulCount'}).get_text()
            except:
                dic['help_count'] = None
            try:
                dic['big_category_name'] = big_category_name
            except:
                dic['big_category_name'] = None
            try:
                dic['category_name'] = category_name
            except:
                dic['category_name'] = None
            try:
                dic['category_id'] = category_id
            except:
                dic['category_id'] = None
            try:
                dic['category_link'] = category_link
            except:
                dic['category_link'] = None
            review.append(dic)

# 크롤링 된 데이터 저장하기
review_df = pd.DataFrame(review)
review_df.to_csv('data/reviews/reviews_{}.csv'.format(cate_name), encoding='utf-8')

#### 4. 셔츠/남방(남) + 셔츠(공)

In [ ]:
cate_name1 = '셔츠/남방'
cate_name = '셔츠'
cate_ids_shirts_M = category_ids[category_ids['category_name'] == cate_name1]
cate_ids_shirts_C = category_ids[category_ids['category_name'] == cate_name]
cate_ids_shirts = cate_ids_shirts_M + cate_ids_shirts_C
cate_ids_shirts

### Product ID 구하기

params 정리  
- listSize -> 60 혹은 120  
- page -> 페이지수  
- sorter -> saleCountDesc(판매량순)  

In [8]:
category_ids.head()

,big_category_name,category_name,category_id
0,쿠팡 홈여성패션의류,후드집업/집업류,498724
1,쿠팡 홈남성패션의류,후드집업/집업류,498929
2,쿠팡 홈남여공용의류,후드집업/집업류,503001


In [141]:
base_url = 'https://www.coupang.com/np/categories/{}'

data = []

for index, row in tqdm(category_ids.iterrows()):
    category_id = row['category_id']
    
    for page in range(1,6):
        print('[{}-{}] category의 {}page 상품 ID 크롤링 시작'.format(row['big_category_name'],row['category_name'],page))
        params = {
            'listSize' : 120,
            'page' : page,
            'sorter' : 'saleCountDesc'
        }
        try:
            res = requests.get(base_url.format(category_id), headers=headers, params=params)
            soup = BeautifulSoup(res.text,'html.parser')
            prod_id_li = eval(soup.find('ul', id='productList')['data-products'])['indexes']

            for prod_id in prod_id_li:
                dic = {}
                try:
                    dic['big_category_name'] = row['big_category_name']
                except:
                    dic['big_category_name'] = None
                try:
                    dic['category_name'] = row['category_name']
                except:
                    dic['category_name'] = None
                try:
                    dic['category_id'] = row['category_id']
                except:
                    dic['category_id'] = None
                try:
                    dic['link'] = row['link']
                except:
                    dic['link'] = None
                try:
                    dic['prod_id'] = prod_id
                except:
                    dic['prod_id'] = None
                data.append(dic)
        except:
            pass



0it [00:00, ?it/s]

[쿠팡 홈여성패션의류-티셔츠] category의 1page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-티셔츠] category의 2page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-티셔츠] category의 3page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-티셔츠] category의 4page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-티셔츠] category의 5page 상품 ID 크롤링 시작


1it [00:07,  7.03s/it]

[쿠팡 홈여성패션의류-맨투맨/후드티] category의 1page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-맨투맨/후드티] category의 2page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-맨투맨/후드티] category의 3page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-맨투맨/후드티] category의 4page 상품 ID 크롤링 시작
[쿠팡 홈여성패션의류-맨투맨/후드티] category의 5page 상품 ID 크롤링 시작


2it [00:14,  7.20s/it]

...

67it [07:54,  8.03s/it]

[쿠팡 홈남녀 공용 의류커플룩/패밀리룩-커플룩/패밀리룩] category의 1page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류커플룩/패밀리룩-커플룩/패밀리룩] category의 2page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류커플룩/패밀리룩-커플룩/패밀리룩] category의 3page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류커플룩/패밀리룩-커플룩/패밀리룩] category의 4page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류커플룩/패밀리룩-커플룩/패밀리룩] category의 5page 상품 ID 크롤링 시작


68it [08:00,  7.56s/it]

[쿠팡 홈남녀 공용 의류빅사이즈-빅사이즈] category의 1page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류빅사이즈-빅사이즈] category의 2page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류빅사이즈-빅사이즈] category의 3page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류빅사이즈-빅사이즈] category의 4page 상품 ID 크롤링 시작
[쿠팡 홈남녀 공용 의류빅사이즈-빅사이즈] category의 5page 상품 ID 크롤링 시작


69it [08:07,  7.06s/it]


In [16]:
review_df.to_csv(r'C:\Users\Aytekin\Desktop\review_reliability\후드집업_model\후드집업_review_data.csv', encoding='utf-8')

In [146]:
df = pd.DataFrame(data)
df

,big_category_name,category_name,category_id,link,prod_id
0,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705,6321766904
1,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705,6750877527
2,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705,6774464085
3,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705,274758307
4,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705,4876745607
...,...,...,...,...,...
40795,쿠팡 홈남녀 공용 의류빅사이즈,빅사이즈,509288,https://www.coupang.com/np/categories/509288,6090096722
40796,쿠팡 홈남녀 공용 의류빅사이즈,빅사이즈,509288,https://www.coupang.com/np/categories/509288,2005490073
40797,쿠팡 홈남녀 공용 의류빅사이즈,빅사이즈,509288,https://www.coupang.com/np/categories/509288,4588348613
40798,쿠팡 홈남녀 공용 의류빅사이즈,빅사이즈,509288,https://www.coupang.com/np/categories/509288,2175968691


In [174]:
# json으로 크롤링 된 데이터 저장하기
df.to_json('product_ids.json', force_ascii=False, orient='records')

In [3]:
# json으로 크롤링 된 데이터 가져오기
with open('product_ids.json', encoding='utf-8') as f:
    temp = json.load(f)
temp = pd.DataFrame(temp)
temp

,big_category_name,category_name,category_id,link,prod_id
0,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705,6321766904
1,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705,6750877527
2,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705,6774464085
3,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705,274758307
4,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705,4876745607
...,...,...,...,...,...
40795,쿠팡 홈남녀 공용 의류빅사이즈,빅사이즈,509288,https://www.coupang.com/np/categories/509288,6090096722
40796,쿠팡 홈남녀 공용 의류빅사이즈,빅사이즈,509288,https://www.coupang.com/np/categories/509288,2005490073
40797,쿠팡 홈남녀 공용 의류빅사이즈,빅사이즈,509288,https://www.coupang.com/np/categories/509288,4588348613
40798,쿠팡 홈남녀 공용 의류빅사이즈,빅사이즈,509288,https://www.coupang.com/np/categories/509288,2175968691


### 리뷰 크롤링

https://www.coupang.com/vp/product/reviews?productId=133071001&page=1&size=5&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=3&ratingSummary=true         

In [4]:
df = temp
# df = df[:1]
df

,big_category_name,category_name,category_id,link,prod_id
0,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705,6321766904
1,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705,6750877527
2,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705,6774464085
3,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705,274758307
4,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705,4876745607
...,...,...,...,...,...
40795,쿠팡 홈남녀 공용 의류빅사이즈,빅사이즈,509288,https://www.coupang.com/np/categories/509288,6090096722
40796,쿠팡 홈남녀 공용 의류빅사이즈,빅사이즈,509288,https://www.coupang.com/np/categories/509288,2005490073
40797,쿠팡 홈남녀 공용 의류빅사이즈,빅사이즈,509288,https://www.coupang.com/np/categories/509288,4588348613
40798,쿠팡 홈남녀 공용 의류빅사이즈,빅사이즈,509288,https://www.coupang.com/np/categories/509288,2175968691


In [10]:
base_url = 'https://www.coupang.com/vp/product/reviews'
review = []

for index, row in tqdm(df.iterrows()):

    big_category_name = row['big_category_name']
    category_name = row['category_name']
    category_id = row['category_id']
    category_link = row['link']
    prod_id = row['prod_id']

    for page in tqdm(range(1,21)):
        params = {
            'productId' : prod_id,
            'page' : page,
            'size' : 30,
            'sortBy' : 'ORDER_SCORE_ASC',
            'viRoleCode' : 3,
            'ratingSummary' : 'true'
        }
        
        res = requests.get(base_url,headers=headers, params=params)
        soup = BeautifulSoup(res.text, 'html.parser')
        datas = soup.find_all('article',{'class':'sdp-review__article__list js_reviewArticleReviewList'})
        for data in datas:
            dic = {}
            try:
                dic['review'] = data.find('div',{'class':'sdp-review__article__list__review__content js_reviewArticleContent'}).get_text(strip=True)
            except:
                continue
            try:
                dic['review_headline'] = data.find('div',{'class':'sdp-review__article__list__headline'}).get_text(strip=True)
            except:
                dic['review_headline'] = None            
            try:
                dic['reviewer'] = data.find('div',{'class':'sdp-review__article__list__info__user'}).get_text(strip=True)
            except:
                dic['reviewer'] = None
            try:
                dic['prod_name'] = data.find('div',{'class':'sdp-review__article__list__info__product-info__name'}).get_text(strip=True)
            except:
                dic['prod_name'] = None
            try:
                dic['prod_id'] = prod_id
            except:
                dic['prod_id'] = None
            try:
                dic['prod_link'] = 'https://www.coupang.com/vp/products/{}'.format(prod_id)
            except:
                dic['prod_link'] = None
            try:
                dic['prod_star_score'] = data.find('div',{'class':'sdp-review__article__list__info__product-info__star-orange js_reviewArticleRatingValue'})['data-rating']
            except:
                dic['prod_star_score'] = None
            try:
                dic['prod_reg_date'] = data.find('div',{'class':'sdp-review__article__list__info__product-info__reg-date'}).get_text(strip=True)
            except:
                dic['prod_reg_date'] = None
            try:
                dic['attach_count'] = len(data.find_all('div',{'class':'sdp-review__article__list__attachment__list'}))
            except:
                dic['attach_count'] = None
            try:
                dic['help_count'] = data.find('span',{'class':'js_reviewArticleHelpfulCount'}).get_text()
            except:
                dic['help_count'] = None
            try:
                dic['big_category_name'] = big_category_name
            except:
                dic['big_category_name'] = None
            try:
                dic['category_name'] = category_name
            except:
                dic['category_name'] = None
            try:
                dic['category_id'] = category_id
            except:
                dic['category_id'] = None
            try:
                dic['category_link'] = category_link
            except:
                dic['category_link'] = None
            review.append(dic)

# 크롤링 된 데이터 저장하기
review_df = pd.DataFrame(review)
review_df.to_excel('review_data.xlsx', encoding='utf-8')

100%|██████████| 20/20 [00:02<00:00,  7.14it/s]
2it [00:06,  3.11s/it]

: 

: 

In [ ]:
with open('review_data.json', encoding='utf-8') as f:
    temp = json.load(f)
temp

In [9]:
df = pd.DataFrame(temp)

In [10]:
df.shape

(278748, 14)

In [15]:
df[:1]

,review,review_headline,reviewer,prod_name,prod_id,prod_link,prod_star_score,prod_reg_date,attach_count,help_count,big_category_name,category_name,category_id,category_link
0,운동할 때 입을 티셔츠 필요하다고 해서 주문해줬어요~편하게 입을 수 있는 반팔 티셔...,운동복으로 좋아요!,다람쥬,"블랙몬스터핏 무지 기능성 쿨에버 헬스 반팔 티셔츠 3P, 2XL, 블랙+화이트+차콜",6321766904,https://www.coupang.com/vp/products/6321766904,5,2023.01.06,5,2,쿠팡 홈여성패션의류,티셔츠,498705,https://www.coupang.com/np/categories/498705


---

총 278748개 데이터 크롤링


columns 정보
-  
       - 'review', 
       - 'review_headline', 
       - 'reviewer', 
       - 'prod_name', 
       - 'prod_id',
       - 'prod_link', 
       - 'prod_star_score', 
       - 'prod_reg_date', 
       - 'attach_count',
       - 'help_count', 
       - 'big_category_name', 
       - 'category_name', 
       - 'category_id',
       - 'category_link'